In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import numpy as np
from flows import Flow
from flows.affine import Planar, Radial
tfd = tfp.distributions
tf.compat.v1.enable_eager_execution()

ModuleNotFoundError: No module named 'base_transform'

In [2]:
k = 2
n_layers = 15
prelu = PReLU()
planar = Planar(k)
radial = Radial(k)
transform = planar
for i in range(n_layers):
    transform = planar(prelu(transform))

In [3]:
base_dist = tfd.MultivariateNormalDiag(loc=tf.zeros((k,)))

In [4]:
flow = Flow(base_dist, transform, learning_rate=1.0E-5, name='affine_gaussian')

AttributeError: 'Chain' object has no attribute 'trainable_variables'

In [ ]:
x1_samples = tfd.Normal(loc=2.0, scale=4).sample(1000)
x2_samples = tfd.Normal(loc=0.5*x1_samples**2.0, scale=2.0*tf.ones(x1_samples.shape)).sample()
x_samples = tf.stack([x2_samples, x1_samples], axis=1)

In [ ]:
plt.scatter(x_samples[:,0], x_samples[:,1])

In [ ]:
flow_dist_samples = flow.dist.sample(512)
plt.scatter(flow_dist_samples[:,0], flow_dist_samples[:,1])

In [ ]:
from flows.affine.planar2 import Planar as Planar2
print(x_samples[:1])
y = planar.forward(x_samples[:1])
print(y)
print(planar.inverse(y))
planar2 = Planar2()
y2, logdet = planar2.forward(x_samples[:1], tf.expand_dims(planar.u, axis=0), tf.expand_dims(planar.w, axis=0), tf.expand_dims(planar.b, axis=0))
print(y2)
print(planar.forward_log_det_jacobian(x_samples[:1], event_ndims=1))
print(logdet)
print(planar.inverse_log_det_jacobian(y, event_ndims=1))

In [ ]:
from tqdm import tqdm_notebook as tqdm
for i in tqdm(range(100000)):
    x_batch = x_samples[i*100 % len(x_samples-100):(i+1)*100%len(x_samples)]
    log_loss, grads = flow.train_on_batch(x_samples)
    if i % 100 == 0:
        #vars_grads = list(zip(map(flow.trainable_variables), map(lambda g: g.numpy(), grads)))
        print(f'(iteration {i}) log loss: {log_loss.numpy()}')
    if i % 1000 == 0:
        samples = flow.dist.sample(512)
        plt.scatter(samples[:,0], samples[:,1])
        plt.show()